# Practica 3
## Maestria en Computo Aplicado

*Topicos de la Industria I*

**Nombre:** Efren Del Real Frias  
**e-mail:** efren.delreal8824@alumnos.udg.mx  
**Fecha:** Abril 15 del 2022

# Software Requirements

* Funciones que generen trayectorias tipo Brownian Motion (BM), Correlated Random Walk (CRW) y Lefy Flight (LF).
Cada una de las funcioones deberá tomar como parámetros el numero de pasos, la velocidad y posición inicial. Además la funciones para CRW  y LF deberán tomar como parámetro el coeficiente para la distribución Cauchy. Por último, la función para LF también deberá aceptar como parámetro el exponente Levy (alpha).
La función retornará un DataFrame de Pandas con todas los puntos que forman la trayectoria.

* El dashboard deberá contener al menos los siguientes elementos (Para generar el dashboard, el estudiante podrá hacer uso de la API de su elección):
* Un panel para desplegar la trayectoria en 3D
* Un panel para desplegar la métrica de elección de la trayectoria.
* Radio Buttons (u otro tipo de selector) que permita elegir el tipo de trayectoria a análizar.
* Widgets que permitan introducir los valores enteros o de punto flotante para ajustar los parámetros de las trayectorias.
Drop down menu (u otro tipo de selector) que permita elegir la métrica a calcualr de la trayectoria bajo análisis.

# Python Header

## Modules

In [123]:
import math
import pandas as pd
import numpy as np

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

import plotly.graph_objects as go

import panel as pn
pn.extension('plotly')

## Classes

In [124]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Functions

In [125]:
def bm_2d(n_steps=1000, speed=6, s_pos=[0, 0]):
  """
  """
  vVelocity = Vec2d(speed, 0)
  vBM_2d_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos':s_pos[1]}])

  for i in range(n_steps - 1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)

    vVelocity = vVelocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{
        'x_pos': vBM_2d_df.x_pos[i] + vVelocity.x,
        'y_pos': vBM_2d_df.y_pos[i] + vVelocity.y}])
    
    vBM_2d_df =pd.concat([vBM_2d_df, temp_df], ignore_index=True)
  
  return vBM_2d_df

In [126]:
###############################################################################################
# Turning angle
###############################################################################################
def turning_angle(vec_a, vec_b, vec_c):
    """
    Arguments:
        vec_a: First detection coordinates
        vec_b: Second detection coordinates
        vec_c: Third detection coordinates
    Returns:
        theta: Turning angle 
    """
    ab = vec_b-vec_a
    norm_ab = np.linalg.norm(ab)
    
    bc = vec_c-vec_b
    norm_bc = np.linalg.norm(bc)

    dot_p = np.dot(ab, bc)
    
    cross_p = np.cross(ab, bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1
    
    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

$$
Euclidean(A, B) = \sqrt{(x_{2} - x_{1})^2  + (y_{2} - y_{1})^2}
$$

$where:$  


*   $A(x_{1}, y_{1})$
*   $B(x_{2}, y_{2})$




In [127]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
def compute_path_lenght(rw_2d_df):
  """
  Arguments:
        rw_2d_df: pd.DataFrame - Random Walk pandas DataFrame size(rows = n, columns = 2)

  Returns:
        pl:       np.array     - Path Lenght numpy array size(rows = n, columns = 1)

  See:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html
  """
  disEuclidean = np.array([ distance.euclidean( rw_2d_df.iloc[i - 1], rw_2d_df.iloc[i])
                            for i in range(1, rw_2d_df.shape[0]) ])
  pl = np.cumsum(disEuclidean)

  return pl

### 3 mean_squared_displacement

$$
MSD = \frac{1}{N - n} \sum_{i=1}^{N-n}(r_{i+n} -r_{i})^2
$$

where $n = 1,...., N - 1$

In [128]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
def mean_squared_displacement(rw_2d_df, tau: int) -> float:
  """
  Arguments:
        rw_2d_df: pd.DataFrame - Random Walk pandas DataFrame size(rows = n, columns = 2)
        tau:      integer      - iterable number to displace two points (called "n" in above equation)
  Returns:
        msd:      float        - mean squared displacement value
  """
  displacement = np.array([ distance.euclidean( rw_2d_df.iloc[i - tau], rw_2d_df.iloc[i] ) 
                            for i in range(tau, rw_2d_df.shape[0]) ])
  
  msd: float = np.mean(np.power(displacement, 2))
  
  return msd

In [129]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
def levy_compute_step_lenght(levy_df):
  """
  Arguments:
        levy_df:                    pd.DataFrame - 
        Levy Walk pandas DataFrame size(rows = n, columns = 2)

  Returns:
        (aux_ta_Levy, aux_sl_Levy):  Tuple(np.array, np.array) - 
        Levy angle and Levy step lengh 
  """
  # Define Local Variables
  theta:        float = 0.0        
  prev_theta:   float = (3 * np.pi)  #
  prev_index:     int = 0
  cal_vel:       bool = True         # Calculate Velocity
  velocity:     float = 0.0

  # aux to store turning angles
  aux_ta_Levy = np.empty(shape=(0))
  # aux to store step-lengths
  aux_sl_Levy = np.empty(shape=(0))

  # Start to calculate over all DataFrame row values
  for index, _ in levy_df[1:-1].iterrows():

      a = levy_df.iloc[index - 1]
      b = levy_df.iloc[index]
      c = levy_df.iloc[index + 1]

      theta = turning_angle(a, b, c)
      
      # Ignore 0.0 since is still in the same direction
      if (theta != 0.0):

        if (prev_theta !=  theta):
          
          # Add to the end the new direction
          aux_ta_Levy = np.concatenate( (aux_ta_Levy, [theta]), axis=0)

          # Gets path lenght - This value includes velocity
          pl = distance.euclidean( levy_df.iloc[prev_index] , levy_df.iloc[index] )

          # Add to the end new path lenght
          pl = np.round(pl)
          aux_sl_Levy = np.concatenate ( (aux_sl_Levy, [pl]) , axis=0)


          # Keeps current index and angle to the next step
          prev_index = index
          prev_theta = theta
      else:
        # theta == 0.0
        if cal_vel:
          # Gets velocity from two close steps
          velocity = distance.euclidean( levy_df.iloc[prev_index] , levy_df.iloc[index] )
          
          # Clean in order to only execute one time
          cal_vel = False

  
  if not cal_vel:
    # We need to divite over the velocity in order
    # to get Levy step lenght
    aux_sl_Levy = aux_sl_Levy / velocity

  return aux_ta_Levy, aux_sl_Levy

## Widgets

In [130]:
wgTrajType = pn.widgets.RadioButtonGroup(
    name='Trajectory type', options=['BM', 'CRW', 'LF'], 
    button_type='default' # [default, primary, success, warning, danger]
    )


wgTrajType

RadioButtonGroup(name='Trajectory type', options=['BM', 'CRW', 'LF'], value='BM')

In [131]:
wgMtcrType = pn.widgets.Select(value='PL', options=['PL','MSD'], name='Metrics type')
wgMtcrType

Select(name='Metrics type', options=['PL', 'MSD'], value='PL')

In [132]:
wgNsteps = pn.widgets.IntSlider(name='number of steps', start=50, end=1000, step=50, value=200)

wgNsteps

IntSlider(end=1000, name='number of steps', start=50, step=50, value=200)

In [133]:
wgSpeed = pn.widgets.IntSlider(name='Speed', start=1, end=10, step=1, value=4)

wgSpeed

IntSlider(end=10, name='Speed', start=1, value=4)

In [134]:
wgXInitPos = pn.widgets.IntInput(name='xInitPos', value=1, step=1, start=0, end=100)
wgXInitPos

IntInput(end=100, name='xInitPos', start=0, value=1)

In [135]:
wgYInitPos = pn.widgets.IntInput(name='yInitPos', value=0, step=1, start=0, end=100)
wgYInitPos

IntInput(end=100, name='yInitPos', start=0)

In [136]:
@pn.depends(wgXInitPos, wgYInitPos)
def display_InitPos(xVal, yVal):
  return (xVal, yVal)

In [137]:
pn.Row(wgXInitPos, wgYInitPos, display_InitPos)

Row
    [0] IntInput(end=100, name='xInitPos', start=0, value=1)
    [1] IntInput(end=100, name='yInitPos', start=0)
    [2] ParamFunction(function)

# Implementation

In [138]:
# Init eGererate Random Walk DataFrame
df_genrw = pd.DataFrame([{'x_pos':  [], 'y_pos': []}])


@pn.depends(wgTrajType, wgNsteps, wgSpeed, wgXInitPos, wgYInitPos)
def main_plotTraj(
    trajType: str,
    Nsteps:   int, 
    speedVal: int, 
    xPos:     int, 
    yPos:     int):
  """
  """

  #
  global df_genrw

  #
  figTraj = go.Figure()

  #
  if ('BM' == trajType):
    pass
  elif ('CRW' == trajType):
    pass
  elif ('LF' == trajType):
    pass

  #
  df_genrw = bm_2d(n_steps=Nsteps, speed=speedVal , s_pos=[xPos, yPos])

  # Plot trajectory in 3D space
  figTraj.add_trace(
      go.Scatter3d(
          x=df_genrw.x_pos,
          y=df_genrw.y_pos,
          z=df_genrw.index,
          marker=dict(size=2),
          line=dict(color='blue', width=2),
          mode='lines',
          name=f'steps = {Nsteps}',
          showlegend=True
      )
  )

  return figTraj

In [139]:
type(df_genrw)

pandas.core.frame.DataFrame

In [140]:
@pn.depends(wgMtcrType)
def main_plotMtcr(mtcrType: str):

  #
  global df_genrw

  #
  metric = np.empty(shape=(0))
  fig_mctr = go.Figure()



  #  
  if ('PL' == mtcrType):
    metric   = compute_path_lenght(df_genrw)
  elif ('MSD' == mtcrType):
    for tau in range(1,df_genrw.shape[0]):
      metric = np.concatenate( (metric, [mean_squared_displacement(df_genrw, tau)]), axis=0)



  # Plot trajectory in 3D space
  fig_mctr.add_trace(
      go.Scatter(
          x=np.arange(len(metric)),
          y=metric,
          marker=dict(size=2),
          line = dict(width=2),
          mode = 'lines',
          name = f'{mtcrType}',
          showlegend = True
  ))

  return fig_mctr

In [141]:
# Dashboard
wgPanel = pn.Row(
    pn.Column(
        'A',
        wgTrajType, 
        wgNsteps,
        pn.Row(wgXInitPos, wgYInitPos),
        wgSpeed),
    pn.Column(
        '3D Trajectory',
        main_plotTraj),
    pn.Column(
        'Metrics',
        wgMtcrType,
        main_plotMtcr)
)
wgPanel

Row
    [0] Column
        [0] Markdown(str)
        [1] RadioButtonGroup(name='Trajectory type', options=['BM', 'CRW', 'LF'], value='BM')
        [2] IntSlider(end=1000, name='number of steps', start=50, step=50, value=200)
        [3] Row
            [0] IntInput(end=100, name='xInitPos', start=0, value=1)
            [1] IntInput(end=100, name='yInitPos', start=0)
        [4] IntSlider(end=10, name='Speed', start=1, value=4)
    [1] Column
        [0] Markdown(str)
        [1] ParamFunction(function)
    [2] Column
        [0] Markdown(str)
        [1] Select(name='Metrics type', options=['PL', 'MSD'], value='PL')
        [2] ParamFunction(function)